Try to use these to fix outputs issue:

https://ipython.readthedocs.io/en/stable/interactive/magics.html
https://notebook.community/lifeinoppo/littlefishlet-scode/RES/REF/python_sourcecode/ipython-master/examples/IPython%20Kernel/Capturing%20Output

# Load SEED Dataset

### Load RAW EEG

In [ ]:
from dataset_processing.seed_dataset_loader import SeedDatasetLoader

sampling_frequency = 200  # 200 Hz

_loader = SeedDatasetLoader(fs=sampling_frequency)

In [ ]:
labels = _loader.get_labels()
labels

In [ ]:
channel_order = _loader.get_channel_order()
channel_order

In [ ]:
_eeg_data_df = _loader.get_eeg_data_df()

In [ ]:
_loader.plot_random_eeg()

In [ ]:
del _loader

### Data Augmentation

In [ ]:
from dataset_processing.eeg_augmentation import EEGAugmentation

_augmentor = EEGAugmentation(_eeg_data_df)
_augmented_df = _augmentor.augment_data()
del _augmentor, _eeg_data_df

# Pre-Training

In [ ]:
from torch.utils.data import DataLoader
from dataset_processing.eeg_dataset import EEGDataset
from model.pre_training.do_pre_training import PreTraining

# From the paper
pretraining_batch_size = 256

In [ ]:
# TODO: Add in bachelor thesis how `num_workers` was chosen with code from w_testing_values notebook

# Custom cleanup function, useful when using the dataloader too much,
# as it's bugged and needs manual cleaning (because of Jupyter Notebook)
def cleanup_data_loader(loader):
    # noinspection PyProtectedMember
    if loader._iterator is not None:
        # noinspection PyProtectedMember
        loader._iterator._shutdown_workers()

##### Results for choosing best value for `num_workers` (HH:MM:SS.milliseconds)
Time/epoch with `num_workers = 0`: `0:18:18.900`

Time/epoch with `num_workers = 1`: `0:08:41.793`

Time/epoch with `num_workers = 2`: `0:06:42.313`

Time/epoch with `num_workers = 3`: `0:06:36.799`

Time/epoch with `num_workers = 4`: `0:06:52.586`

Time/epoch with `num_workers = 8`: `0:06:44.983`

### Dataset Loader

In [ ]:
try:
    # noinspection PyUnresolvedReferences,PyUnboundLocalVariable
    _dataset
except NameError:
    # noinspection PyUnboundLocalVariable
    _dataset = EEGDataset(_augmented_df)
    del _augmented_df

In [ ]:
num_workers = 3

data_loader = DataLoader(
    _dataset,
    batch_size=pretraining_batch_size,
    shuffle=True,
    pin_memory=True,
    persistent_workers=True,

    num_workers=num_workers,
    prefetch_factor=2,  # Default: 2 for `num_workers` > 0  # TODO: Maybe set to 3-4
)
# del _augmented_df, _dataset

### Do the pre-training

In [ ]:
# from importlib import reload
# import model.pre_training.do_pre_training
# 
# reload(model.pre_training.do_pre_training)
# 
# PreTraining = model.pre_training.do_pre_training.PreTraining

In [ ]:
# TODO: Use for simplified training: https://pytorch-ignite.ai/tutorials/beginner/01-getting-started/

In [ ]:
print(f"Starting pre-training with {num_workers} workers loading the dataset")

pretraining = PreTraining(
    data_loader=data_loader,
    sampling_frequency=sampling_frequency,
    pretraining_model_save_dir="model_params/pretraining",
    scheduler_patience=50,
    early_stopping_patience=100,
    epochs=2000,
)
pretraining.train()

cleanup_data_loader(data_loader)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs

# Fine-Tuning

### Dataset Loader

In [ ]:
# from torch.utils.data import DataLoader
# from dataset_processing.eeg_dataset import EEGDataset
# 
# # From the paper
# pretraining_batch_size = 256
# 
# _dataset = EEGDataset(_augmented_df)
# data_loader = DataLoader(_dataset, batch_size=pretraining_batch_size, shuffle=True)
# del _augmented_df, _dataset

### Do the fine-tuning

In [ ]:
# from model.fine_tuning.do_fine_tuning import FineTuning
# 
# finetuning = FineTuning(
#     data_loader=data_loader,
#     sampling_frequency=sampling_frequency,
#     finetuning_model_save_dir="model_params/finetuning",
#     epochs=10,
# )
# finetuning.train()

# Ideas

Do a correlation matrix between the channels of the EEG signals.
Then when doing the joint whatever model, use the "distances" between the channels (like the hamming distance but not really), as a "weight" for training the joining etc.

Or maybe just output something that could show each channel's contribution towards the final emotion prediction.